In [177]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [179]:
#pip install beautifulsoup4


In [6]:
import requests
import re
from bs4 import BeautifulSoup


In [181]:
#Request Soup 형성




In [8]:
class UserPostData:
    def __init__(self, user, user_id, title, date, posting_index):
        self.user = user
        self.user_id = user_id
        self.title = title
        self.date = date
        self.posting_index = int(posting_index)

    def get_user(self):
        return self.user
    def get_user_id(self): 
        return self.user_id
    def get_date(self):
        return self.date
    def get_data_index(self):
        return self.posting_index
    def get_title(self):
        return self.title[1:]
    
class UserPostDataList:
    def __init__(self):
        self.user_list = []
        self.user_id_list = []
        self.title_list = []
        self.date_list = []
        self.posting_index_list = []
        #self.user_dict = {}
        #self.user_posting_dict = {}
        self.earliest_date = ""
        self.latest_date = ""
        self.min_posting_id = 0
        self.max_posting_id = 0
        self.length = 0

    def add_user_post_data(self, user_post):
        self.user_list.append(user_post.get_user())
        self.user_id_list.append(user_post.get_user_id())
        self.title_list.append(user_post.get_title())
        self.date_list.append(user_post.get_date())
        self.posting_index_list.append(user_post.get_data_index())
        self.length += 1
        return self
    
    def get_earliest_and_latest_date(self):
        if self.date_list == []:
            self.earliest_date = "1999-01-01 00:00:00"
            self.latest_date = "1999-01-01 23:39:39"
            return self
        else:
            self.earliest_date = self.date_list[self.length-1]
            self.latest_date = self.date_list[0]    
            return self

    def get_earliest_and_latest_posting(self):
        if self.posting_index_list == []:
            self.min_posting_id = 0
            self.max_posting_id = 0
            return self
        else:
            self.min_posting_id = self.posting_index_list[self.length-1]
            self.max_posting_id = self.posting_index_list[0]
            return self

    def compare(self, target_time, debug = False):
        if debug == True:
            print("Comparing " + str(target_time) + " with bound " + str(self.earliest_date) + " and " + str(self.latest_date))
        if target_time >= self.earliest_date and target_time <= self.latest_date:
            return 0
        elif target_time < self.earliest_date:
            return -1
        else:
            return 1  

    def __add__(self, other_datalist, debug = False): #Adding UserPoistDataList
        if self.length == 0:
            self.user_list = other_datalist.user_list
            self.user_id_list = other_datalist.user_id_list
            self.title_list = other_datalist.title_list
            self.date_list = other_datalist.date_list
            self.posting_index_list = other_datalist.posting_index_list
            self.length = other_datalist.length
            self.earliest_date = other_datalist.earliest_date
            self.latest_date = other_datalist.latest_date
            self.min_posting_id = other_datalist.min_posting_id
            self.max_posting_id = other_datalist.max_posting_id
            return self
        cut = 0
        if debug == True:
            print("Adding UserPostDataList of length " +str(other_datalist.length))
            print("current min_posting id: " + str(self.min_posting_id))
            print("other min_posting id: " + str(other_datalist.min_posting_id))
            print("current max_posting id: " + str(self.max_posting_id))
            print("other max_posting id: " + str(other_datalist.max_posting_id))
            
        for i in range(other_datalist.length):
            if other_datalist.posting_index_list[i] >= self.min_posting_id:
                cut = i+1
                print("We are cutting " + str(cut))
            else:
                break
            
        if cut == 0:
            print("No cutting")
        self.user_list = self.user_list + other_datalist.user_list[cut:]
        self.user_id_list = self.user_id_list + other_datalist.user_id_list[cut:]
        self.title_list = self.title_list + other_datalist.title_list[cut:]
        self.date_list = self.date_list + other_datalist.date_list[cut:]
        self.posting_index_list = self.posting_index_list + other_datalist.posting_index_list[cut:]
        self.length = self.length + other_datalist.length - cut
        self.min_posting_id = other_datalist.min_posting_id
        self.earliest_date = other_datalist.earliest_date
        return self
    
    def get_user_post_data_list(self):
        return self.user_post_data_list

    def __str__(self, min = 0, max = 0, get_content = False):
        print("Earliest Date: " + str(self.earliest_date) + ", Latest Date: " + str(self.latest_date))
        print("Min Posting ID: " + str(self.min_posting_id) + ", Max Posting ID: " + str(self.max_posting_id) + ", Length: " + str(self.length))
        if get_content == True:
            if max == 0:
                max = self.length

            for i in range(min, max):
                print("Title: " + str(self.title_list[i]))
                print("ID: " + str(self.user_list[i]) + ", " + str(self.user_id_list[i]))
                print("Index: " + str(i) + ", posting_index: " + str(self.posting_index_list[i]) + ", Date:" + str(self.date_list[i]))
                print()

        #return "UserPostDat: " + str(self.user_list) + ", " + str(self.user_id_list) + ", " + str(self.title_list) + ", " + str(self.date_list) + ", " + str(self.posting_index_list)

        return ""

class UserPostDataDict:
    def __init__(self, UserPostList):
        self.userPostList = UserPostList
        self.user_dict = {}
        self.user_posting_dict = {}

    def match_user_id_to_user_name(self):
        for i in range(self.userPostList.length):
            if self.userPostList.user_id_list[i] in self.user_dict:
                if self.userPostList.user_list[i] not in self.user_dict[self.userPostList.user_id_list[i]]:
                    self.user_dict[self.userPostList.user_id_list[i]].append(self.userPostList.user_list[i])
            else:
                self.user_dict[self.userPostList.user_id_list[i]] = [self.userPostList.user_list[i]]
        return self
    
    def collect_user_activities_by_user_id(self):
        for user_id in self.userPostList.user_id_list:
            if user_id in self.user_posting_dict:
                self.user_posting_dict[user_id] += 1
            else:
                self.user_posting_dict[user_id] = 1
        return self
    
    def build(self):
        self.match_user_id_to_user_name()
        self.collect_user_activities_by_user_id()
        return self

    def search_user_id_by_user(self, user):
        for key in self.user_dict:
            if user in self.user_dict[key]:
                return key
        return -1
    
    def search_user_posing_count(self,user):
        user_id = self.search_user_id_by_user(user)
        if user_id == -1:
            return 0
        else:
            return self.user_posting_dict[user_id]

    def __str__(self):
        print("User Dictionary: " + str(self.user_dict))
        print("User Posting Dictionary: " + str(self.user_posting_dict))
        return ""


In [30]:
import random
import time
user_agents = [
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
    # Add more user agents as needed
]

# Randomly select a User-Agent

def extractSoupfromURL(url):
    random_user_agent = random.choice(user_agents)
    print(random_user_agent)
    headers = {
        'User_Agent': random_user_agent
    }
    request = requests.get(url, headers=headers)
    soup = BeautifulSoup(request.content, 'html.parser')
    time_to_sleep = random.uniform(1, 3)
    time.sleep(time_to_sleep)
    print(time_to_sleep)
    return request, soup

def construct_url(gall_url, page_num):
    return gall_url + "&page=" + str(page_num)

In [32]:
from datetime import datetime, timedelta

#시간 범위 확인
def is_date_in_range(date_str, start_date_str, end_date_str):
    date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    if len(start_date_str) == 10:
        start_date_str = start_date_str + " 00:00:00"
    if len(end_date_str) == 10:
        end_date_str = end_date_str + " 23:59:59"
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M:%S')
    end_date = end_date + timedelta(days=1, seconds=-1) # to 23:59:59
    return start_date <= date <= end_date

#공지 걸러내는 용도
def is_Notification(soup):
    if soup.find('td', class_ = "gall_subject").text == "공지":
        #print("공지 발견")
        return True
    else:
        return False

#한 줄에서 user, user_id, date를 추출
def get_userPostData_from_line(sample):
    try:
        user = sample.find('span', class_ = "nickname in").text
        user_id = sample.find('td', class_ = "gall_writer ub-writer")['data-uid']
    except:
        title1 = sample.find('span', class_ = "nickname").text
        title2 = sample.find('span', class_ = "ip").text
        user = title1 + title2
        user_id = user
    #print(user)
    #print(user_id)
    title = sample.find('a').text
    #print(title[1:])
    date_raw = sample.find('td', class_ = "gall_date")
    date = date_raw['title']
    #print(date)
    posting_index = sample.find('td', class_ = "gall_num").text
    userPostData = UserPostData(user, user_id, title, date, posting_index)
    return userPostData

#특정 페이지에서 start_date, end_date내의 user, user_id의 리스트 추출

def get_user_and_date_from_soup_list(soup_list, start_date = "", end_date = ""):
    userPostDataList = UserPostDataList()
    #date_list = []
    for line in soup_list:
        if is_Notification(line) is False:
            UserPostData = get_userPostData_from_line(line)
            if start_date != "" and end_date != "":
                if is_date_in_range(UserPostData.get_date(), start_date, end_date):
                    UserPostData = get_userPostData_from_line(line)
                    userPostDataList.add_user_post_data(UserPostData)
            else:
                UserPostData = get_userPostData_from_line(line)
                userPostDataList.add_user_post_data(UserPostData)
    userPostDataList.get_earliest_and_latest_date()
    userPostDataList.get_earliest_and_latest_posting()
    return userPostDataList


def get_soup_list_from_page(gall_name, page_num):
    url = construct_url(gall_name, page_num)
    print(url)
    request, soup = extractSoupfromURL(url)
    print(request.text)
    soup_list = soup.find_all('tr', class_ = 'ub-content us-post')
    return request, soup_list

def concat_userPostDataList(userPostDataList_list):
    userPostDataList = UserPostDataList()
    for userPostDataList_append in userPostDataList_list:
        userPostDataList = userPostDataList + userPostDataList_append
    return userPostDataList



#parse_user_list(user_id_list)
    

In [33]:

request, soup_list = get_soup_list_from_page("https://gall.dcinside.com/mini/board/lists/?id=pumpitup", 1)
request1, soup_list2 = get_soup_list_from_page("https://gall.dcinside.com/mini/board/lists/?id=pumpitup", 2)
request2, soup_list3 = get_soup_list_from_page("https://gall.dcinside.com/mini/board/lists/?id=pumpitup", 3)





https://gall.dcinside.com/mini/board/lists/?id=pumpitup&page=1
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36
1.4335511175506228

https://gall.dcinside.com/mini/board/lists/?id=pumpitup&page=2
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36
2.9596731771942117

https://gall.dcinside.com/mini/board/lists/?id=pumpitup&page=3
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36
2.389169155513251



In [35]:
sample_userPostDataList1 = get_user_and_date_from_soup_list(soup_list, '2024-03-31', '2024-04-02')
sample_userPostDataList2 = get_user_and_date_from_soup_list(soup_list2, '2024-03-31', '2024-04-02')
sample_userPostDataList3 = get_user_and_date_from_soup_list(soup_list3, '2024-03-31', '2024-04-02')


#
print("Take1: ")
print(sample_userPostDataList1)
print("Take2: ")
print(sample_userPostDataList2)
print("Take3: ")
print(sample_userPostDataList3)
#print("SUM: ")
#print(sample_userPostDataList)
 #'2024-03-31', '2024-04-02'

Take1: 
Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0

Take2: 
Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0

Take3: 
Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0



In [26]:
print(sample_userPostDataList1)
print(sample_userPostDataList2)
print(sample_userPostDataList3)

sample2 = concat_userPostDataList([sample_userPostDataList1, sample_userPostDataList2, sample_userPostDataList3])

print(sample_userPostDataList1)
print(sample_userPostDataList2)
print(sample_userPostDataList3)
print(sample2)

Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0

Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0

Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0

Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0

Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0

Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0

Earliest Date: 1999-01-01 00:00:00, Latest Date: 1999-01-01 23:39:39
Min Posting ID: 0, Max Posting ID: 0, Length: 0



In [22]:
userPostDataDict = UserPostDataDict(sample2)
userPostDataDict.build()

print(userPostDataDict)

userPostDataDict.search_user_id_by_user("채은서")
#userPostDataDict.search_user_posing_count("앙기무띠")


User Dictionary: {}
User Posting Dictionary: {}



-1

In [27]:
import time
import random
#유저 ID별 활동량을 저장함
def collect_user_activities_by_user_id(user_id_dict, user_list):
    #user_dict = {}
    for user in user_list:
        #print(user_dict)
        if user in user_id_dict:
            user_id_dict[user] += 1
        else:
            user_id_dict[user] = 1
    return user_id_dict


def do_binary_search_page(gall_name, min_page, max_page, target_date):
    if max_page < min_page:
        return -1
    mid_page = (max_page + min_page) // 2
    print("iterating:" + str(mid_page))
    is_soup_found = False
    while(is_soup_found == False):
        soup_list = get_soup_list_from_page(gall_name, str(mid_page))
        sample_data = get_user_and_date_from_soup_list(soup_list, target_date, target_date)
        print(sample_data.length)
        if sample_data.length != 0:
            is_soup_found = True
    print()
    compare = sample_data.compare(target_date, debug = True)
    time_to_sleep = random.uniform(1, 3)
    time.sleep(time_to_sleep)
    if compare == 0:
        return mid_page
    elif compare == -1:
        return do_binary_search_page(gall_name, mid_page+1, max_page, target_date)
    else:
        return do_binary_search_page(gall_name, min_page, mid_page-1, target_date)


def set_start_end_page(gall_name, max_page, min_page, start_date, end_date):
    get_start_date_page = do_binary_search_page(gall_name, min_page, max_page, start_date)
    get_end_date_page = do_binary_search_page(gall_name, min_page, max_page, end_date)
    return get_start_date_page, get_end_date_page



In [28]:
do_binary_search_page("https://gall.dcinside.com/mini/board/lists/?id=pumpitup", 1, 52, '2024-03-31')

iterating:26
https://gall.dcinside.com/mini/board/lists/?id=pumpitup&page=26
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15
2.752196879104025
0
https://gall.dcinside.com/mini/board/lists/?id=pumpitup&page=26
Mozilla/5.0 (iPad; CPU OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1
2.2404173640813996
0
https://gall.dcinside.com/mini/board/lists/?id=pumpitup&page=26
Mozilla/5.0 (iPad; CPU OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1


KeyboardInterrupt: 

In [110]:
i = construct_url("https://gall.dcinside.com/mini/board/lists/?id=pumpitup", 1680)

print(i)

soup_blank = get_soup_list_from_page("https://gall.dcinside.com/mini/board/lists/?id=pumpitup", 1680)

get_user_and_date_from_soup_list(soup_blank, '2014-03-31', '2024-04-02')


https://gall.dcinside.com/mini/board/lists/?id=pumpitup&page=1680


([], [], 0, '1999-01-01 00:00:00', '1999-01-01 00:00:00')

In [100]:
#고정닉 닉변 대응 용도
def match_user_id_to_user_name(user_dict, user_list, user_id_list):
    for i in range(len(user_list)):
        if user_id_list[i] in user_dict:
            if user_list[i] not in user_dict[user_id_list[i]]:
                user_dict[user_id_list[i]].append(user_list[i])
        else:
            user_dict[user_id_list[i]] = [user_list[i]]
    return user_dict

users = ["A", "B", "C", "D", "E", "F", "G"]
user_id = ["1", "2", "3", "4", "1", "3", "5"]

user_id_dict= {}

user_dict_updated = match_user_id_to_user_name(user_id_dict, users, user_id)

user2 = ["H", "I", "J", "K", "L", "M", "N"]
user_id2 = ["1", "2", "3", "4", "1", "3", "5"]


match_user_id_to_user_name(user_dict_updated, user2, user_id2)


{'1': ['A', 'E', 'H', 'L'],
 '2': ['B', 'I'],
 '3': ['C', 'F', 'J', 'M'],
 '4': ['D', 'K'],
 '5': ['G', 'N']}

In [2]:
import time
import random

# Sleep for a random time between 1 to 3 seconds



def do_binary_search(start, end, target):
    if start > end:
        return -1
    mid = (start + end) // 2
    time_to_sleep = random.uniform(1, 3)
    time.sleep(time_to_sleep)
    print(time_to_sleep)
    print(mid)
    if target == mid:
        return mid
    elif target < mid:
        return do_binary_search(start, mid-1, target)
    else:
        return do_binary_search(mid+1, end, target) 

do_binary_search(0, 100, 16)


2.845194638064627
50
2.430273848089227
24
2.6839733240909185
11
1.5811300152178827
17
2.0703910674937323
14
2.9662980535930554
15
1.696629885585466
16


16